# Floreano Experiment in the NRP

## Initialize the Virtual Coach with Storage Server credentials

In [ ]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In [ ]:
import os
import sys
import time
import pandas
import shutil
import plot_utils
import evolution_utils

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach

%matplotlib inline
vc = VirtualCoach(environment='local', storage_username='nrpuser', storage_password='password')

## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [ ]:
display_episode_tf = """@nrp.Robot2Neuron()
def display_episode_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

Run the evolutionary experiment after specifying the number of generations and individuals per generation

In [ ]:
class FloreanoExperiment(object):
    
    def __init__(self, experiment_name, population, generations):
        self.experiment_name = experiment_name
        self.experiment_dir = os.environ['HOME'] + '/.opt/nrpStorage/' + self.experiment_name
        self.last_status = [None]
        self.fitness_log = []
        self.sim = None
        self.generations = generations
        
        # Check if evolutionary experiment has been run before
        # If yes, load previously evolved population and continue
        previous_generations = [s for s in os.listdir(self.experiment_dir) if "generation" in s]
        previous_generations.sort()
        if len(previous_generations) == 0:
            self.population = population
            self.cur_gen = 0
        else:
            last_gen_dir = self.experiment_dir + '/' + previous_generations[-1]
            # current generation number
            self.cur_gen = int(previous_generations[-1].split('_')[1])
            individuals = [s for s in os.listdir(last_gen_dir) if "individual" in s]
            # If generation has not been fully simulated, delete it and start from the previous one
            if len(individuals) > 60:
                self.population = evolution_utils.evolve_new_generation(last_gen_dir)
                self.cur_gen += 1
            else:
                shutil.rmtree(last_gen_dir)
                # If there's only one incomplete generation, start over
                if self.cur_gen < 0:
                    self.population = population
                else:
                    last_gen_dir = self.experiment_dir + '/generation_{}'.format((self.cur_gen - 1)) 
                    self.population = evolution_utils.evolve_new_generation(last_gen_dir)

    def wait_for_localhost(self, timeout):
        """
        helper method that waits for the localhost server to be available
        again after stopping a simulation
        """
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(2)
            if 'localhost' in vc.print_available_servers():
                return
        raise Excpetion('Cannot find Localhost')

    def wait_condition(self, timeout, condition):
        """
        Helper method that blocks for the timeout specified, until the condition
        given has been fulfilled
        """
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(1)
            if self.last_status[0] is not None:
                if condition(self.last_status[0]):
                    return
        raise Exception('Condition check failed')

    def on_status(self, status):
        """Prepends the most recent ROS status message to the last_status array"""
        self.last_status[0] = status
        
    def save_simulation_data(self, generation, trial):
        """
        Saves the simulation csv data to the respective individual's directory inside the
        experiment's directory
        """
        self.sim.save_csv()
        csv_dir = [s for s in os.listdir(self.experiment_dir) if "csv_records" in s][0]
        individual_dir = self.experiment_dir + '/generation_{}'.format(generation) + '/individual_{}'.format(trial)
        shutil.move(self.experiment_dir + '/' + csv_dir, individual_dir)
        np.save(individual_dir + '/genetic_string', self.population[trial])

        wheel_speeds = evolution_utils.get_wheel_speeds(individual_dir)
        np.save(individual_dir + '/wheel_speeds', wheel_speeds)
        trajectory = evolution_utils.get_trajectory(individual_dir)
        collision = evolution_utils.correct_for_collisions(individual_dir)

        # save fitness value
        if collision:
            fitness_value = -1
        else:
            fitness_value = evolution_utils.fitness_function(wheel_speeds)
        print "Fitness: {}".format(fitness_value)
        np.save(individual_dir + '/fitness_value', fitness_value)


    def run_experiment(self):
        # launch experiment and register status callback
        self.sim = vc.launch_experiment('floreano_0', server='localhost')
        self.sim.register_status_callback(self.on_status)

        for i in range(self.cur_gen, self.generations):    
            # Create directory for generation data
            os.mkdir(self.experiment_dir + '/generation_{}'.format(i))
            
            # Iterate over individuals in a population
            for j in range(60):
                clear_output(wait=True)
                print "Generation {}, Individual {}".format(i, j)
                genetic_string = ','.join(str(x) for x in self.population[j].ravel())
                self.sim.edit_brain(evolution_utils.brain % genetic_string)
                #self.sim.add_transfer_function(display_episode_tf % "Generation {}, Individual {}".format(i, j))
                self.sim.start()
                
                # run simulation for 40 seconds
                self.wait_condition(10000, lambda x: x['simulationTime'] > 40)
                self.sim.pause()
                self.save_simulation_data(i, j)
                start = time.time()
                self.sim.reset('full')
                print "Reset Time: {}".format(time.time() - start)
                self.wait_condition(1000, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)

            evolution_utils.get_top_performers(self.experiment_dir + '/generation_{}'.format(i))
            self.population = evolution_utils.evolve_new_generation(self.experiment_dir + '/generation_{}'.format(i))

In [ ]:
population = np.random.randint(2, size=(60, 10, 29)) # random population of 10 binary genetic strings
floreano_experiment = FloreanoExperiment('floreano_0', population, 30)
floreano_experiment.run_experiment()

## Plot the saved csv data and save the plots

In [ ]:
plot_utils.plot_spikes(individual_dir)
plot_utils.plot_trajectory(individual_dir, trajectory)
plot_utils.plot_wheel_speeds(individual_dir, wheel_speeds)